_한국어로 기계번역됨_


# 서브그래프에서 스트림하기

!!! 정보 "전제 조건"

    이 가이드는 다음에 대한 이해를 전제로 합니다:
    
    - [서브그래프](../../concepts/low_level/#subgraphs)
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)

[서브그래프](../subgraph)를 사용하여 그래프를 생성했다면, 해당 서브그래프에서 출력 스트리밍을 원할 수 있습니다. 이렇게 하려면 부모 그래프의 `.stream()` 메서드에서 `subgraphs=True`를 지정할 수 있습니다:


```python
for chunk in parent_graph.stream(
    {"foo": "foo"},
    # 다음 줄 강조
    subgraphs=True
):
    print(chunk)
```

## 설정

먼저 필요한 패키지를 설치합시다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위해 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 빠르게 발견하고 성능을 향상시킬 수 있습니다. LangSmith를 사용하면 LangGraph로 만든 LLM 앱을 디버그하고, 테스트하고, 모니터링하기 위해 추적 데이터를 활용할 수 있습니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>에서 확인하세요. 
    </p>
</div>


## 예시


간단한 예제를 정의해 보겠습니다:


In [2]:
from langgraph.graph import START, StateGraph
from typing import TypedDict


# Define subgraph
class SubgraphState(TypedDict):
    foo: str  # note that this key is shared with the parent graph state
    bar: str


def subgraph_node_1(state: SubgraphState):
    return {"bar": "bar"}


def subgraph_node_2(state: SubgraphState):
    return {"foo": state["foo"] + state["bar"]}


subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")
subgraph = subgraph_builder.compile()


# Define parent graph
class ParentState(TypedDict):
    foo: str


def node_1(state: ParentState):
    return {"foo": "hi! " + state["foo"]}


builder = StateGraph(ParentState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", subgraph)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
graph = builder.compile()


이제 그래프에서 출력을 스트리밍해 보겠습니다.


In [3]:
for chunk in graph.stream({"foo": "foo"}, stream_mode="updates"):
    print(chunk)


{'node_1': {'foo': 'hi! foo'}}
{'node_2': {'foo': 'hi! foobar'}}


부모 그래프 노드(`node_1`와 `node_2`)에서만 업데이트를 내보내고 있음을 알 수 있습니다. _서브그래프_ 노드의 업데이트를 내보내려면 `subgraphs=True`를 지정하면 됩니다.


In [4]:
for chunk in graph.stream(
    {"foo": "foo"},
    stream_mode="updates",
    # highlight-next-line
    subgraphs=True,
):
    print(chunk)


((), {'node_1': {'foo': 'hi! foo'}})
(('node_2:b692b345-cfb3-b709-628c-f0ba9608f72e',), {'subgraph_node_1': {'bar': 'bar'}})
(('node_2:b692b345-cfb3-b709-628c-f0ba9608f72e',), {'subgraph_node_2': {'foo': 'hi! foobar'}})
((), {'node_2': {'foo': 'hi! foobar'}})


자! 스트리밍 된 출력에는 이제 부모 그래프와 하위 그래프의 업데이트가 모두 포함됩니다. **주의**: 우리는 노드 업데이트뿐만 아니라 어떤 그래프(또는 하위 그래프)에서 스트리밍되고 있는지를 알려주는 네임스페이스도 받고 있습니다.
